In [8]:
from newspaper import Article
from transformers import pipeline


def chunk_text(text, max_chunk_size=1024):
    sentences = text.split('. ')
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_chunk_size:
            current_chunk += sentence + ". "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "

        chunks.append(current_chunk.strip())

    return chunks


def extract_article(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except Exception as e:
        print(f"An error occurred while extracting the article: {e}")
        return None

def summarize_article(article_text):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


    chunks = chunk_text(article_text)
    summary = []

    for chunk in chunks:
        input_length = len(chunk.split())
        max_length = min(150, input_length - 1)

        if input_length < 50:
            max_length = 30
            min_length = 10

        try:
            chunk_summary = summarizer(chunk, max_length=max_length, min_length=int(max_length/2), do_sample=False)
            summary.append(chunk_summary[0]['summary_text'])
        except Exception as e:
            print(f"An error occurred during summarization: {e}")
            continue

    return "\n\n".join(summary).strip()

def count_words(text):
    return len(text.split())

def summarize_news_article(url):
    article_text = extract_article(url)
    if article_text:
        original_word_count = count_words(article_text)
        print(f"Original article word count: {original_word_count} words")

        print("Article extracted successfully! Summarizing the article now...\n")
        summary = summarize_article(article_text)

        if summary:
            summarized_word_count = count_words(summary)
            print(f"Summarized article word count: {summarized_word_count} words")
            print("Summary:\n", summary)
        else:
            print("Failed to summarize the article.")
    else:
        print("Failed to extract the article.")


if __name__ == "__main__":
    url = input("Enter the news article URL: ")
    summarize_news_article(url)


Enter the news article URL: https://indianexpress.com/article/political-pulse/in-maharashtra-sharad-pawar-has-last-laugh-as-mahayuti-leaders-make-a-beeline-to-join-his-party-9567474/?ref=hometop_hp
Original article word count: 580 words
Article extracted successfully! Summarizing the article now...

Summarized article word count: 234 words
Summary:
 Five senior leaders from the BJP and the Ajit Pawar-led NCP have crossed over to the NCP (SP) since April. On Thursday, NCP leader Bhagyashree Atram, the daughter of an NCP minister, switched to Sharad Pawar’s party. She said her father was giving false assurance of welfare to the public.

The NCP (SP) bagged eight of the 10 seats it contested. The overall tally of the Opposition Maha Vikas Aghadi (MVA) was 30 out of 48 seats. The ruling Mahayuti alliance of the BJP, Chief Minister Eknath Shinde-led Shiv Sena, and the NCP could manage to get only 17 seats.

Among Mahayuti partners, the NCP proved to be the worst performer, winning only one 